In [85]:
import boto3
import json
import os
from rich import print
from mypy_boto3_cloudwatch import CloudWatchClient
from mypy_boto3_cloudwatch.type_defs import GetDashboardOutputTypeDef


DASHBOARD_NAME = "FilesAPIDashboard"

cloudwatch_client: "CloudWatchClient" = boto3.client("cloudwatch")

## Dashboard

In [158]:
# cloudwatch_client.get_dashboard(DashboardName="FilesAPIDashboard")

In [159]:
# response: "GetDashboardOutputTypeDef" = cloudwatch_client.get_dashboard(DashboardName=DASHBOARD_NAME)

# dashboard_body: str | None = response.get("DashboardBody")
# dashboard_body: dict = json.loads(dashboard_body)
# dashboard_body.keys()

In [160]:
# from datetime import datetime, timezone

# # "2024-07-04T09:37:08.000Z"
# datetime.now(timezone.utc).isoformat(timespec="milliseconds").replace("+00:00", "Z")

In [139]:
from pathlib import Path
from datetime import datetime, timezone

VERSION_TXT_PATH = Path("../version.txt")


def get_deployment_verison(verion_txt_path: Path) -> str:
    """Get the deployment version from version.txt"""
    if not verion_txt_path.exists():
        raise FileNotFoundError(f"File not found: {verion_txt_path}")

    # read version.txt
    with open(verion_txt_path, "r") as f:
        version = f.read().strip()
    return version


def add_vertical_annotations_to_widgets(dashboard_body: dict, version_txt_path: Path, N: int) -> dict:
    """
    Add a vertical annotation to all timeSeries widgets in the dashboard with the current deployment version.

    Args:
        :dashboard_body: dict: The dashboard body from CloudWatch
        :version_txt_path: Path: The path to the version.txt file
        :N: int: The number of deployment events to keep for the annotations
    Returns:
        :dict: The updated dashboard body
    """
    # get deployment version
    version = get_deployment_verison(version_txt_path)

    # get current time in ISO format, "2024-07-04T09:37:08.000Z"
    current_time = datetime.now(timezone.utc).isoformat(timespec="milliseconds").replace("+00:00", "Z")

    for widget in dashboard_body.get("widgets", []):
        widget_properties: dict = widget.get("properties") or dict()
        if widget_properties.get("view") == "timeSeries":
            # get annotations
            annotations: dict[str, list[dict]] = widget_properties.get("annotations") or dict()

            # add vertical annotation
            vertical_annotations = annotations.get("vertical") or list()

            # only keep last n deployments annotations
            vertical_annotations = vertical_annotations[-N:]
            vertical_annotations.append({"color": "#69ae34", "label": f"{version}", "value": current_time})

            # overwrite current annotations
            annotations["vertical"] = vertical_annotations
            widget_properties["annotations"] = annotations

    return dashboard_body


def reset_dashboard_widgets(dashboard_body: dict) -> dict:
    """
    Reset the dashboard widgets by removing all vertical annotations.

    Args:
        :dashboard_body: dict: The dashboard body from CloudWatch
    Returns:
        :dict: The updated dashboard body
    """
    for widget in dashboard_body.get("widgets", []):
        widget_properties: dict = widget.get("properties") or dict()
        if widget_properties.get("view") == "timeSeries":
            # get annotations
            annotations: dict[str, list[dict]] = widget_properties.get("annotations") or dict()
            vertical_annotations = annotations.get("vertical") or list()
            if vertical_annotations:
                annotations["vertical"] = []  # remove vertical annotations

            widget_properties["annotations"] = annotations

    return dashboard_body

In [140]:
# dashboard_body

In [141]:
# reset_dashboard_widgets(dashboard_body)

In [142]:
def update_dashboard_widgets(
    cloudwatch_client: "CloudWatchClient" = None,
    dashboard_name: str = None,
    version_txt_path: Path = None,
    N: int = None,
    reset: bool = False,
) -> None:
    """
    Update the dashboard widgets with vertical line for the deployment version.

    args:
        :cloudwatch_client: CloudWatchClient: The CloudWatch client
        :dashboard_name: str: The name of the dashboard
        :verion_txt_path: Path: The path to the version.txt file
        :N: int: The number of deployment events to keep for the vertical annotations, default 10
        :reset: bool: Reset the dashboard widgets, i.e. remove all vertical annotations; default False

    returns: None
    """
    version_txt_path = version_txt_path or VERSION_TXT_PATH
    N = N or 10
    dashboard_name = dashboard_name or os.environ["DASHBOARD_NAME"]
    cloudwatch_client = cloudwatch_client or boto3.client("cloudwatch")
    response: "GetDashboardOutputTypeDef" = cloudwatch_client.get_dashboard(DashboardName=dashboard_name)

    dashboard_body: str | None = response.get("DashboardBody")
    dashboard_body: dict = json.loads(dashboard_body)

    if reset:
        # reset dashboard widgets
        updated_dashboard_body = reset_dashboard_widgets(dashboard_body)

    # update dashboard widgets
    updated_dashboard_body = add_vertical_annotations_to_widgets(dashboard_body, version_txt_path, N=N)

    # update the dashboard
    cloudwatch_client.put_dashboard(
        DashboardName=dashboard_name,
        DashboardBody=json.dumps(updated_dashboard_body),
    )

In [143]:
DASHBOARD_NAME = "FilesAPIDashboard"
VERSION_TXT_PATH = Path("../version.txt")

update_dashboard_widgets(dashboard_name=DASHBOARD_NAME, version_txt_path=VERSION_TXT_PATH, reset=True)

## Alarms

In [152]:
import boto3
from mypy_boto3_sns import SNSClient
from mypy_boto3_sns.type_defs import CreateTopicResponseTypeDef

# https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/sns.html
# https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/sns/client/create_topic.html
def create_sns_topic(topic_name: str, sns_client: "SNSClient" = None) -> str:
    """
    Create a new SNS topic or get the TopicARN if it already exists.

    Args:
        :sns_client: SNSClient: The SNS client
        :topic_name: str: The name of the topic

    Returns:
        :str: The ARN of the new/existing topic
    """
    sns_client = sns_client or boto3.client("sns")
    response: "CreateTopicResponseTypeDef" = sns_client.create_topic(Name=topic_name)
    print(f"Created SNS topic Response: {response}")
    topic_arn = response["TopicArn"]
    return topic_arn


# https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/sns/client/subscribe.html
def subscribe_email_to_topic(topic_name: str, email: str, sns_client: "SNSClient" = None) -> None:
    """
    Subscribe an email to an SNS topic.

    Args:
        :sns_client: SNSClient: The SNS client
        :topic_name: str: The name of the SNS topic to subscribe to. If the topic does not exist, it will be created.
        :email: str: The email to subscribe

    Returns:
        :None:
    """
    sns_client = sns_client or boto3.client("sns")
    topic_arn = create_sns_topic(topic_name)
    response = sns_client.subscribe(TopicArn=topic_arn, Protocol="email", Endpoint=email)
    print(f"Email Subscription Response: {response}")
    print("Please confirm the subscription by clicking the link in the email sent to you.")


demo_topic_name = "demo-topic"
subscribe_email_to_topic(topic_name=demo_topic_name, email="avr13405@gmail.com")

Created SNS topic Response: {'TopicArn': 'arn:aws:sns:ap-south-1:730335491176:demo-topic', 'ResponseMetadata': 
{'RequestId': 'cf97e0f2-b70a-55b1-a58b-bfeed0830d2a', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': 
'cf97e0f2-b70a-55b1-a58b-bfeed0830d2a', 'date': 'Thu, 25 Jul 2024 14:46:33 GMT', 'content-type': 'text/xml', 
'content-length': '318', 'connection': 'keep-alive'}, 'RetryAttempts': 0}}

Email Subscription Response: {'SubscriptionArn': 'pending confirmation', 'ResponseMetadata': {'RequestId': 
'4d3ece02-98a5-5189-a115-911eff2c1b35', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': 
'4d3ece02-98a5-5189-a115-911eff2c1b35', 'date': 'Thu, 25 Jul 2024 14:46:33 GMT', 'content-type': 'text/xml', 
'content-length': '298', 'connection': 'keep-alive'}, 'RetryAttempts': 0}}

Please confirm the subscription by clicking the link in the email sent to you.

In [156]:
import boto3
from mypy_boto3_cloudwatch import CloudWatchClient

# https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/cloudwatch.html

# https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/cloudwatch/client/put_metric_alarm.html

def create_latency_alarm(
    alarm_name: str,
    sns_topic_arn: str = None,
    cloudwatch_client: "CloudWatchClient" = None,
) -> None:
    """
    Create a CloudWatch alarm for the Files API latency metric.

    Args:
        :alarm_name: str: The name of the alarm
        :sns_topic_arn: str: The SNS topic ARN to send the alarm notification
        :cloudwatch_client: CloudWatchClient: The CloudWatch client
    """
    alarm_config = {
        "AlarmName": alarm_name,
        "MetricName": "Latency",
        "Namespace": "AWS/ApiGateway",
        "ExtendedStatistic": "p95",
        "Period": 60,  # 1 minute
        "EvaluationPeriods": 3,  # N=3, The number of periods over which data is compared to the specified threshold.
        "DatapointsToAlarm": 2,  # M=2 out of N=3 datapoints must be breaching the threshold
        "Threshold": 5000,  # 5 seconds
        "ComparisonOperator": "GreaterThanThreshold",
        "TreatMissingData": "notBreaching",
        "AlarmDescription": "If the p95 Latency > 3000 milliseconds of Files API endpoints constrained to the evaluation period then the alarm is triggered.",
        "Dimensions": [{"Name": "ApiName", "Value": "Files API"}],
    }

    if sns_topic_arn:
        alarm_config["AlarmActions"] = [sns_topic_arn]

    cloudwatch_client = cloudwatch_client or boto3.client("cloudwatch")
    cloudwatch_client.put_metric_alarm(**alarm_config)


def create_4xx_errors_alarm(
    alarm_name: str,
    sns_topic_arn: str = None,
    cloudwatch_client: "CloudWatchClient" = None,
) -> None:
    """
    Create a CloudWatch alarm for the Files API 4xx errors metric.
    
    Args:
        :alarm_name: str: The name of the alarm
        :sns_topic_arn: str: The SNS topic ARN to send the alarm notification
        :cloudwatch_client: CloudWatchClient: The CloudWatch client
    """
    alarm_config = {
        "AlarmName": alarm_name,
        "MetricName": "4XXError",
        "Namespace": "AWS/ApiGateway",
        "Statistic": "Sum",
        "Period": 60,  # 1 minute
        "EvaluationPeriods": 3,
        "DatapointsToAlarm": 2,
        "Threshold": 5,  # 5 errors
        "ComparisonOperator": "GreaterThanThreshold",
        "TreatMissingData": "notBreaching",
        "AlarmDescription": "If the number of 4xx Errors > 5 for any endpoint of FilesAPI constrained to the evaluation period then the alarm is triggered.",
        "Dimensions": [{"Name": "ApiName", "Value": "Files API"}],
    }

    if sns_topic_arn:
        alarm_config["AlarmActions"] = [sns_topic_arn]

    cloudwatch_client = cloudwatch_client or boto3.client("cloudwatch")
    cloudwatch_client.put_metric_alarm(**alarm_config)
    


def create_5xx_errors_alarm(
    alarm_name: str,
    sns_topic_arn: str = None,
    cloudwatch_client: "CloudWatchClient" = None,
) -> None:
    """
    Create a CloudWatch alarm for the Files API 5xx errors metric.
    
    Args:
        :alarm_name: str: The name of the alarm
        :sns_topic_arn: str: The SNS topic ARN to send the alarm notification
        :cloudwatch_client: CloudWatchClient: The CloudWatch client
    """
    alarm_config = {
        "AlarmName": alarm_name,
        "MetricName": "5XXError",
        "Statistic": "Sum",
        "Period": 60,  # 1 minute
        "EvaluationPeriods": 3,
        "DatapointsToAlarm": 2,
        "Threshold": 5,  # 5 errors
        "ComparisonOperator": "GreaterThanThreshold",
        "TreatMissingData": "notBreaching",
        "AlarmDescription": "If the number of 5xx Errors > 5 for any endpoint of FilesAPI constrained to the evaluation period then the alarm is triggered.",
        "Namespace": "AWS/ApiGateway",
        "Dimensions": [{"Name": "ApiName", "Value": "Files API"}],
    }

    if sns_topic_arn:
        alarm_config["AlarmActions"] = [sns_topic_arn]

    cloudwatch_client = cloudwatch_client or boto3.client("cloudwatch")
    cloudwatch_client.put_metric_alarm(**alarm_config)


In [157]:
latency_alarm_name = "DEMO-FilesAPI-Latency-Alarm"
errors_4xx_alarm_name = "DEMO-FilesAPI-4xx-Errors-Alarm"
errors_5xx_alarm_name = "DEMO-FilesAPI-5xx-Errors-Alarm"

sns_topic_arn = create_sns_topic(demo_topic_name)

create_latency_alarm(alarm_name=latency_alarm_name, sns_topic_arn=sns_topic_arn)
create_4xx_errors_alarm(alarm_name=errors_4xx_alarm_name, sns_topic_arn=sns_topic_arn)
create_5xx_errors_alarm(alarm_name=errors_5xx_alarm_name, sns_topic_arn=sns_topic_arn)

Created SNS topic Response: {'TopicArn': 'arn:aws:sns:ap-south-1:730335491176:demo-topic', 'ResponseMetadata': 
{'RequestId': '35988006-30c3-5c42-9d83-df1a58564c79', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': 
'35988006-30c3-5c42-9d83-df1a58564c79', 'date': 'Thu, 25 Jul 2024 14:51:01 GMT', 'content-type': 'text/xml', 
'content-length': '318', 'connection': 'keep-alive'}, 'RetryAttempts': 0}}

<p align="center">
  <img src="../assets/sns-demo.png" alt="SNS Topic Created" />
  <br />
  <em>SNS Topic Created</em>
</p>

<p align="center">
  <img src="../assets/alarms_demo.png" alt="Alarms Created" />
  <br />
  <em>Alarms Created</em>
</p>


In [3]:
import boto3
from mypy_boto3_cloudwatch import CloudWatchClient

def enable_alarm_actions(
    alarm_names: list[str],
    cloudwatch_client: "CloudWatchClient" = None,
) -> None:
    """
    Enable the alarm actions for the specified alarms.

    Args:
        :alarm_names: list[str]: The names of the alarms to enable actions
        :cloudwatch_client: CloudWatchClient: The CloudWatch client
    """
    cloudwatch_client = cloudwatch_client or boto3.client("cloudwatch")
    cloudwatch_client.enable_alarm_actions(AlarmNames=alarm_names)
    

def disable_alarm_actions(
    alarm_names: list[str],
    cloudwatch_client: "CloudWatchClient" = None,
) -> None:
    """
    Disable the alarm actions for the specified alarms.

    Args:
        :alarm_names: list[str]: The names of the alarms to disable actions
        :cloudwatch_client: CloudWatchClient: The CloudWatch client
    """
    cloudwatch_client = cloudwatch_client or boto3.client("cloudwatch")
    cloudwatch_client.disable_alarm_actions(AlarmNames=alarm_names)

In [5]:
latency_alarm_name = "DEMO-FilesAPI-Latency-Alarm"
errors_4xx_alarm_name = "DEMO-FilesAPI-4xx-Errors-Alarm"
errors_5xx_alarm_name = "DEMO-FilesAPI-5xx-Errors-Alarm"

disable_alarm_actions(alarm_names=[latency_alarm_name, errors_4xx_alarm_name])

<p align="center">
  <img src="../assets/disable-alarm-demo.png" alt="Alarms Disabled" />
  <br />
  <em>Alarms Disabled</em>
</p>
